In [4]:
#!/usr/bin/python3
# -*-coding:utf-8
'''
Created on Fri Dec 1 22:22:35 2017

@author: Ray


'''

import time
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import utils # written by author
from glob import glob
from datetime import datetime, timedelta
import multiprocessing as mp
import gc # for automatic releasing memory

In [7]:
##################################################
# Load  
##################################################
input_col = ['msno','transaction_date','is_membership_duration_equal_to_plan_days',
'is_membership_duration_longer_than_plan_days','is_early_expiration']
membership_loyalty = utils.read_multiple_csv('../../input/preprocessed_data/transactions_date_base') # 20,000,000

membership_loyalty = membership_loyalty.head(n = 500)

##################################################
# Convert string to datetime format
##################################################
membership_loyalty['transaction_date']  = membership_loyalty.transaction_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(membership_loyalty)

100%|██████████| 8/8 [00:00<00:00, 2843.84it/s]

reduce memory


In [6]:
membership_loyalty.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 0 to 499
Data columns (total 8 columns):
msno                                            500 non-null object
transaction_date                                500 non-null datetime64[ns]
membership_duration                             500 non-null int16
is_membership_duration_equal_to_plan_days       500 non-null int8
is_membership_duration_longer_than_plan_days    500 non-null int8
days_longer_than_plan_days                      500 non-null int8
is_early_expiration                             500 non-null int8
early_expiration_days                           500 non-null int8
dtypes: datetime64[ns](1), int16(1), int8(5), object(1)
memory usage: 15.1+ KB


In [7]:
def near(x, keep = 5):
    return x.tail(keep)

#==============================================================================
# def
#==============================================================================
def make(T):
	"""
	T = 0
	folder = 'trainW-0'
	"""

	if T ==-1:
	    folder = 'test'
	    train = pd.read_csv('../input/sample_submission_v2.csv') # 此train代表的是test的user
	else:
	    folder = 'trainW-'+str(T)
	    train = pd.read_csv('../input/preprocessed_data/trainW-{0}.csv'.format(T))[['msno']]

	# the following style is silly, but it's all for saving memory 
	if T == 0:
		df = pd.merge(train, 
	    membership_loyalty[(membership_loyalty.transaction_date < datetime.strptime('2017-03-01', '%Y-%m-%d'))], 
	    on=['msno'], 
	    how='left')
		del train
	elif T == 1:
	    # w = 1:使用2月之前的資料當作history
	    df = pd.merge(train, 
	    	membership_loyalty[(membership_loyalty.transaction_date < datetime.strptime('2017-02-01', '%Y-%m-%d'))],
	    	on=['msno'], 
	    	how='left') 
	    del train
	elif T == 2:
	    # w = 2:使用1月之前的資料當作history
	    df = pd.merge(train, 
	    	membership_loyalty[(membership_loyalty.transaction_date < datetime.strptime('2017-01-01', '%Y-%m-%d'))],
	    	on=['msno'], 
	    	how='left') 
	    del train
	elif T == -1:
	    # w = -1:使用4月之前的資料當作history
	    df = pd.merge(train, 
	    	membership_loyalty[(membership_loyalty.transaction_date < datetime.strptime('2017-04-01', '%Y-%m-%d'))],
	    	on='msno', 
	    	how='left') 
	    del train
	##################################################
	# All history
	##################################################
	#df = df.dropna()
	#core
	tbl = df.groupby('msno').membership_duration.mean().to_frame()
	tbl.columns = ['membership_duration-mean']
	tbl['membership_duration-min'] = df.groupby('msno').membership_duration.min()
	tbl['membership_duration-max'] = df.groupby('msno').membership_duration.max()
	tbl['membership_duration-median'] = df.groupby('msno').membership_duration.median()
	tbl['membership_duration-std'] = df.groupby('msno').membership_duration.std()
	tbl['transactions_count'] = df.groupby('msno').membership_duration.size()
	# is_churn: missing
	tbl.reset_index(inplace = True)
	#==============================================================================
	print('reduce memory')
	#==============================================================================
	utils.reduce_memory(tbl)
	#write
	tbl.to_csv('../feature/{}/membership_duration.csv'.format(folder), index = False)
	del tbl
	gc.collect()
	##################################################
	# near 5
	##################################################
	#core
	tbl = df.groupby('msno').apply(near,5).reset_index(drop = True).groupby('msno').membership_duration.mean().to_frame()
	tbl.columns = ['membership_duration-mean_n5']
	tbl['membership_duration-min_n5'] = df.groupby('msno').membership_duration.min()
	tbl['membership_duration-max_n5'] = df.groupby('msno').membership_duration.max()
	tbl['membership_duration-median_n5'] = df.groupby('msno').membership_duration.median()
	tbl['membership_duration-std_n5'] = df.groupby('msno').membership_duration.std()
	tbl.reset_index(inplace = True)
	#==============================================================================
	print('reduce memory')
	#==============================================================================
	utils.reduce_memory(tbl)
	#write
	tbl.to_csv('../feature/{}/membership_duration_n5.csv'.format(folder), index = False)
	del tbl
	gc.collect()
	##################################################
	# only one prvious order
	##################################################
	#core
	tbl = df.groupby('msno').apply(near,1).reset_index(drop = True).groupby('msno').membership_duration.mean().to_frame()
	tbl.columns = ['membership_duration-mean_n1']
	tbl['membership_duration-min_n1'] = df.groupby('msno').membership_duration.min()
	tbl['membership_duration-max_n1'] = df.groupby('msno').membership_duration.max()
	tbl['membership_duration-median_n1'] = df.groupby('msno').membership_duration.median()
	tbl['membership_duration-std_n1'] = df.groupby('msno').membership_duration.std()
	tbl.reset_index(inplace = True)
	#==============================================================================
	print('reduce memory')
	#==============================================================================
	utils.reduce_memory(tbl)
	#write
	tbl.to_csv('../feature/{}/membership_duration_n1.csv'.format(folder), index = False)
	del tbl
	gc.collect()



In [ ]:
##################################################
# Main
##################################################
s = time.time()
make(0)
# make(1)
# make(2)
# make(-1)


In [80]:
T = 0
folder = 'trainW-'+str(T)
train = pd.read_csv('../../input/preprocessed_data/trainW-{0}.csv'.format(T))[['msno']]

In [81]:
df = pd.merge(train, 
membership_loyalty[(membership_loyalty.transaction_date < datetime.strptime('2017-03-01', '%Y-%m-%d'))], 
on=['msno'], 
how='left')


In [82]:
df = df.dropna()

In [41]:
input_col = ['msno','transaction_date','is_membership_duration_equal_to_plan_days',
'is_membership_duration_longer_than_plan_days','is_early_expiration']
df[input_col].head(n = 10)


,msno,transaction_date,is_membership_duration_equal_to_plan_days,is_membership_duration_longer_than_plan_days,is_early_expiration
58581,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2016-06-22,0.0,0.0,0.0
58582,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2016-07-21,0.0,1.0,0.0
58583,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2016-08-21,0.0,1.0,0.0
58584,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2016-09-21,1.0,0.0,0.0
58585,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2016-10-21,0.0,1.0,0.0
58586,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2016-11-21,1.0,0.0,0.0
58587,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2016-12-21,0.0,1.0,0.0
58588,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2017-01-21,0.0,1.0,0.0
58589,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,2017-02-21,0.0,0.0,0.0
86379,++4RuqBw0Ss6bQU4oMxaRlbBPoWzoEiIZaxPM04Y4+U=,2015-01-13,0.0,1.0,0.0


In [42]:
df[df.msno == '+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=']

,msno,transaction_date,membership_duration,is_membership_duration_equal_to_plan_days,is_membership_duration_longer_than_plan_days,days_longer_than_plan_days,is_early_expiration,early_expiration_days
746867,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,2015-01-26,31.0,0.0,1.0,1.0,0.0,0.0
746868,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,2015-02-26,28.0,0.0,0.0,0.0,0.0,0.0
746869,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,2015-03-26,31.0,0.0,1.0,1.0,0.0,0.0
746870,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,2015-04-26,30.0,0.0,1.0,30.0,0.0,0.0
746871,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,2015-05-26,31.0,0.0,1.0,1.0,0.0,0.0
746872,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,2015-06-26,30.0,1.0,0.0,0.0,0.0,0.0
746873,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,2015-07-26,31.0,0.0,1.0,1.0,0.0,0.0
746874,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,2015-08-26,31.0,0.0,1.0,1.0,0.0,0.0
746875,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,2015-09-26,30.0,1.0,0.0,0.0,0.0,0.0
746876,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,2015-10-26,31.0,0.0,1.0,1.0,0.0,0.0


In [56]:
df.groupby('msno').is_membership_duration_equal_to_plan_days.mean()

msno
+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=    0.000000
+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=    0.000000
+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=    0.269231
++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=    0.333333
++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=    0.000000
++/ZHqwUNa7U21Qz+zqteiXlZapxey86l6eEorrak/g=    0.000000
++0/NopttBsaAn6qHZA2AWWrDg7Me7UOMs1vsyo4tSI=    0.333333
++0BJXY8tpirgIhJR14LDM1pnaRosjD1mdO1mIKxlJA=    0.937500
++0EzISdtKY48Z0GY62jer/LFQwrNIAbADdtU5xStGY=    0.833333
++0wqjjQge1mBBe5r4ciHGKwtF/m322zkra7CK8I+Mw=    0.250000
++1G0wVY14Lp0VXak1ymLhPUdXPSFJVBnjWwzGxBKJs=    0.333333
++1GCIyXZO7834NjDKmcK1lBVLQi9PsN6sOC7wfW+8g=    0.375000
++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=    0.222222
++2Ju1OdxLSyexwhZ/C0glNK0DMIfUjsFpk9lt8Dll8=    0.222222
++38dVm0EHKCOfND6oEal3xFKtqJYZTZJmnpbAOOZBk=    0.294118
++3A6JMzYJeron30GTcDostfXoAl8rTBuB2M8GeVdNU=    0.375000
++3Z+W8OPnpbHYfrKwqRKN1bF83XEbxjdYUolhGdHZg=    0.037037
++3fWHRDC5GWWlovHcrOKWNwZY

In [86]:
tbl = df.groupby(['msno']).is_membership_duration_equal_to_plan_days.sum().to_frame()
tbl.columns = ['is_membership_duration_equal_to_plan_days_cnt']
tbl['is_membership_duration_equal_to_plan_days_ratio'] = df.groupby('msno').is_membership_duration_equal_to_plan_days.mean()
tbl['is_membership_duration_longer_than_plan_days_cnt'] = df.groupby('msno').is_membership_duration_longer_than_plan_days.sum()
tbl['is_membership_duration_longer_than_plan_days_ratio'] = df.groupby('msno').is_membership_duration_longer_than_plan_days.mean()
tbl['is_early_expiration_cnt'] = df.groupby('msno').is_early_expiration.sum()
tbl['is_early_expiration_ratio'] = df.groupby('msno').is_early_expiration.mean()
tbl.reset_index(inplace = True)
tbl

,msno,is_membership_duration_equal_to_plan_days_cnt,is_membership_duration_equal_to_plan_days_ratio,is_membership_duration_longer_than_plan_days_cnt,is_membership_duration_longer_than_plan_days_ratio,is_early_expiration_cnt,is_early_expiration_ratio
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,0.0,0.000000,2.0,0.500000,0.0,0.000000
1,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,0.0,0.000000,20.0,1.000000,0.0,0.000000
2,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,7.0,0.269231,16.0,0.615385,0.0,0.000000
3,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,4.0,0.333333,7.0,0.583333,0.0,0.000000
4,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,0.0,0.000000,20.0,1.000000,0.0,0.000000
5,++/ZHqwUNa7U21Qz+zqteiXlZapxey86l6eEorrak/g=,0.0,0.000000,22.0,1.000000,0.0,0.000000
6,++0/NopttBsaAn6qHZA2AWWrDg7Me7UOMs1vsyo4tSI=,4.0,0.333333,7.0,0.583333,0.0,0.000000
7,++0BJXY8tpirgIhJR14LDM1pnaRosjD1mdO1mIKxlJA=,15.0,0.937500,1.0,0.062500,0.0,0.000000
8,++0EzISdtKY48Z0GY62jer/LFQwrNIAbADdtU5xStGY=,5.0,0.833333,1.0,0.166667,0.0,0.000000
9,++0wqjjQge1mBBe5r4ciHGKwtF/m322zkra7CK8I+Mw=,4.0,0.250000,9.0,0.562500,0.0,0.000000


In [6]:
T = 0
folder = 'trainW-'+str(T)
train = pd.read_csv('../../input/preprocessed_data/trainW-{0}.csv'.format(T))[['msno']]



In [ ]:
def near(x, keep = 5):
    return x.tail(keep)

In [8]:
df = pd.merge(train, 
membership_loyalty[(membership_loyalty.transaction_date < datetime.strptime('2017-03-01', '%Y-%m-%d'))], 
on=['msno'], 
how='left')
del train


In [ ]:
df_ = df.groupby('msno').apply(near,5).reset_index(drop = True)